# Finding Latest Share prices in DOW( week2)

## Introduction

## Data

## Analysis
       First, we import the necessary packages and do some preparation for our analysis. We can retrieve the data from the DOW website.
      

In [9]:
import requests
import re
import pandas as pd

def retrieve_dji_list():
    try:
        r = requests.get('https://money.cnn.com/data/dow30/')
    except ConnectionError as err:
        print(err) 
    search_pattern = re.compile('class="wsod_symbol">(.*?)<\/a>.*?<span.*?">(.*?)<\/span>.*?\n.*?class="wsod_stream">(.*?)<\/span>')
    dji_list_in_text = re.findall(search_pattern, r.text)
    dji_list = []
    for item in dji_list_in_text:
        dji_list.append([item[0], item[1], float(item[2])])
    return dji_list
 
dji_list = retrieve_dji_list()
djidf = pd.DataFrame(dji_list)
print(djidf)

       0                        1       2
0    MMM                       3M  163.10
1    AXP         American Express  116.76
2   AAPL                    Apple  236.41
3     BA                   Boeing  344.00
4    CAT              Caterpillar  130.71
5    CVX                  Chevron  114.74
6   CSCO                    Cisco   46.71
7     KO                Coca-Cola   54.78
8    DIS                   Disney  130.89
9    DOW             Dow Chemical   47.96
10   XOM              Exxon Mobil   67.61
11    GS            Goldman Sachs  206.52
12    HD               Home Depot  237.93
13   IBM                      IBM  134.09
14  INTC                    Intel   51.36
15   JNJ        Johnson & Johnson  127.70
16   JPM           JPMorgan Chase  120.56
17   MCD               McDonald's  208.50
18   MRK                    Merck   84.68
19  MSFT                Microsoft  137.41
20   NKE                     Nike   96.10
21   PFE                   Pfizer   36.46
22    PG         Procter & Gamble 

Second, we rename some indext so that it looks more readable.

In [10]:
cols=['code','name','lasttrade']
djidf.columns=cols # 改变列名
djidf.index=range(1,len(djidf)+1)

djidf.code   # 获取code列+index
djidf['code']  # 获取code列 , 两者同功能
djidf.loc[1:5,]  # 前5行
djidf.loc[:,['code','lasttrade']] #所有行
djidf.loc[1:5,['code','lasttrade']] #1-5行, loc表示标签index
djidf.loc[1,['code','lasttrade']] #1行
djidf.at[1,'lasttrade']  # 只有一个值的时候可以用at
djidf.iloc[2:4,[0,2]]  # 表示物理文职, 并且4取不到, 就只有第三行第四行
djidf.iat[1,2]  # 单个值


116.76

### 1. Analysis of History Transcation

We can find the company names for the highest share prices.

In [11]:
djidf.lasttrade.mean()  # 121.132
djidf[djidf.lasttrade>=180].name



tempdf=djidf.sort_values(by='lasttrade',ascending=False)
tempdf[:3].name

4           Boeing
26    UnitedHealth
13      Home Depot
Name: name, dtype: object

We can see that, Boeing has the highest share price, and then UnitedHealth, and Home Depot.

### 2. Analysis of AXP comapny

Next, for example we want to know the share price of AXP company in the last year.

In [12]:
import requests
import re
import json
import pandas as pd
from datetime import  date
def retrieve_quotes_historical(stock_code):
    quotes = []
    url = 'https://finance.yahoo.com/quote/%s/history?p=%s' % (stock_code, stock_code)
    try:
        r = requests.get(url)
    except ConnectionError as err:
        print(err)
    m = re.findall('"HistoricalPriceStore":{"prices":(.*?),"isPending"', r.text)
    if m:
        quotes = json.loads(m[0])       # m = ['[{...},{...},...]'] 
        quotes = quotes[::-1]         # 原先数据为date最新的在最前面
    return  [item for item in quotes if not 'type' in item]
quotes = retrieve_quotes_historical('AXP')
list1=[]
for i in range(len(quotes)):
    x=date.fromtimestamp(quotes[i]['date'])
    y=date.strftime(x,'%Y-%m-%d')
    list1.append(y)
quotesdf_ori=pd.DataFrame(quotes,index=list1)    
quotesdf_m = quotesdf_ori.drop(['adjclose'], axis = 1)  #删除adjclose列
quotesdf=quotesdf_m.drop(['date'],axis=1)
print(quotesdf)

                 close        high         low        open   volume
2018-10-22  104.510002  106.959999  104.449997  106.610001  5003100
2018-10-23  104.379997  104.519997  101.839996  102.410004  4223800
2018-10-24  101.839996  104.949997  101.510002  104.430000  4056700
2018-10-25  103.599998  104.169998  101.800003  102.480003  3378900
2018-10-26  101.250000  102.660004  100.139999  102.540001  5395700
2018-10-29  101.190002  103.250000  100.040001  102.470001  4238700
2018-10-30  102.080002  102.389999  100.410004  101.599998  3778200
2018-10-31  102.730003  103.709999  102.550003  103.059998  4511300
2018-11-01  104.040001  104.269997  103.019997  103.260002  2786800
2018-11-02  103.709999  105.050003  102.889999  104.930000  4322200
2018-11-05  105.209999  105.400002  103.800003  104.040001  2697700
2018-11-06  104.980003  105.660004  104.370003  104.980003  2856000
2018-11-07  107.309998  107.480003  104.900002  105.730003  3606900
2018-11-08  108.500000  108.629997  107.029999  

Then we get it ! We can make statistical summary for this company, including the decreasing and increasing days of share price, and description of max, min sd, etc.

In [13]:
quotesdf[(quotesdf.index>='2017-03-01') & (quotesdf.index<='2017-03-31')]
quotesdf[(quotesdf.index>='2017-11-30') & (quotesdf.index<='2018-03-31')& 
(quotesdf.close>=90)]
len(quotesdf.close>quotesdf.open) 

250

In [14]:
import pandas as pd
import numpy as np
index_df = pd.DataFrame(np.random.rand(3,3), index=['a','b','c'], columns=['index_1','index_2','index_3'])
index_df.describe()

,index_1,index_2,index_3
count,3.000000,3.000000,3.000000
mean,0.422109,0.528184,0.271551
std,0.261885,0.136680,0.413067
min,0.163625,0.424300,0.009406
25%,0.289529,0.450765,0.033472
50%,0.415434,0.477231,0.057538
75%,0.551351,0.580126,0.402624
max,0.687268,0.683022,0.747710


In [15]:
Finally, we still want to know the number of transcation days for each month. We can also accomplish this in python.

SyntaxError: invalid syntax (<ipython-input-15-b45ef18deb3b>, line 1)

In [16]:
import time
list2=[]
for i in range(len(quotesdf)):
    temp=time.strptime(quotesdf.index[i],'%Y-%m-%d')
    list2.append(temp.tm_mon)  # 取月份
tempdf=quotesdf.copy()
tempdf['month']=list2  # 新增一列月份的数据
print(tempdf['month'].value_counts())  # 计算每个月的出现次数

10    22
8     22
7     22
5     22
11    21
4     21
3     21
1     21
9     20
6     20
12    19
2     19
Name: month, dtype: int64


## Conlusion

With the analysis above, we are able to summary the transaction features of DOM. Boeing，AmericanHealth, and Home Depot have the higheest share price currently. For specific compaies like AXP, 
we are also able to get lots of valauble infomration about its share price with this analysis. Basically, in this project, we apply method of description statistics and show our results in tables. Admittedly, charts and visualization an enrich the content of this 
analysis, but it does not necessary for share price analysis.